# Streaming - Imagery API

## Overview

The **Streaming - Imagery API** uses the **Open Geospatial Consortium (OGC)** API services **Web Feature Service (WFS)**, **Web Map Service (WMS)** and **Web Map Tile Service (WMTS)** to provide RGB imagery and metadata ready for use in a map-like interface. For example, the Streaming API provides the imagery displayed in the MGP Pro web application and is ideal to use as an imagery layer in GIS software. A single request can return display data showing one or more image products stacked together. In **Streaming Search App** the following endpoints from the API are used:

## WMS Get Map
Generates a map based on the input query parameters. In this case the endpoint will use the legacy identifier property of a feature to know what map area to will the images be displayed. As you zoom in and out of a map, it will bring the images of the map acording to the layer.

## WFS Get Feature
Searches for imagery features in the Maxar online catalog, the feature coordinates are going to be used to display polygons in the interactive map, and the legacy identifier property of a feature to display images within the area of the coordinates.

### Filter Parameters Used in Streaming Search App
In the code bellow, you are able to modify the query of the endpoint by changing the the values of the variables that represent the following parameters.
- **bbox** — `query`  
  Bounding box in format `"south,west,north,east"` in WGS84 decimal degrees, coordinates of the square area were the features are in.

- **aquisitionDate** — `query`  
  Filtered using **Date From** and **Date To** fields in app to create a range of dates were the features where aquired.

- **sunElevation** — `query`  
  Displayed as **Sun Angle** in app, user may only insert an angle between 0-90, features fetched will have less or equal to that angle.

- **cloudCover** — `query`  
  Displayed as **Image Cloud Cover** in app, features will have less or equal of percentage in clouds in images (value between 0-100)

- **offNadirAngle** — `query`  
  Displayed as **Off nadir Angle** in app, features have less or equal to that angle (value between 0-90)

- **sensors** — `query`  
  Comma-separated list of sensors to search features from. If this parameter is not specified, items are retrieved from all available sensors.
---

For more information, please refer to the [API Documentation](https://developers.maxar.com/docs/streaming-imagery/).


In [1]:
import requests
import json
from getpass import getpass

maxar_api_key = getpass("Enter your API Key")

######################
#Filtering Parameters#
######################
date_from='2023-11-25' #Beggining of date range (YYYY-MM-DD)
date_to='2024-11-25' #End of date range (YYYY-MM-DD)
sun_angle=90 #(value between 0-90)
image_cloud_coverage=50 #(value between 0-100)
off_nadir_angle=90 #(value between 0-90)
sensors= ['WV01','WV02', 'WV03', 'WV02_VNIR', 'WV03_SWIR', 'GE01', 'QB02', 'LG01', 'LG02'] #By default all sensors of the endpoint documentation are currently being selected
bbox= [39.483063458085226, -105.2641865080031, 40.069740459067766, -104.64631129972818] #By default is set in the city of Denver

#CQL Filter
cql_filter_parameters = [
    f"(bbox(featureGeometry,{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]})",
    f"(acquisitionDate>={date_from})",
    f"(acquisitionDate<={date_to})",
    f"(sunElevation<={sun_angle})",
    f"(cloudCover<={image_cloud_coverage})",
    f"(offNadirAngle<={off_nadir_angle})",
    f"(source IN {tuple(sensors)}))"
]
cql_filter = " AND ".join(cql_filter_parameters)

#WFS Get Feature Endpoint
url = "https://api.maxar.com/streaming/v1/ogc/wfs"

#Endpoint Parameters
payload = {
    'cql_filter': cql_filter, 
    'count': 1000, 
    'service':'WFS', 
    'request': 'GetFeature', 
    'version':'2.0.0', 
    'typeNames': 'Maxar:FinishedFeature', 
    'outputFormat': 'application/json'
}
headers = {
    'Accept': 'application/geo+json',
    'MAXAR-API-KEY': maxar_api_key
}

#Endpoint is called and features are fetched
response = requests.request("GET", url, headers=headers, params=payload)
response.raise_for_status()
response_json = response.json()

print("Finished recieving data from WFS Get Feature Endpoint, please move to the next cell.")
print(f"Number of images recieved: {response_json['numberReturned']}")

Enter your API Key ········


Finished recieving data from WFS Get Feature Endpoint, please move to the next cell.
Number of images recieved: 301


## Downloading GeoJSON

If you would like to download the Streaming-Imagery metadata, please proceed by running the next code cell to initiate the download process.

_Note_: This will retrieve metadata from the API and save it in the current directory that jupyter was started on. Process might take a few seconds.


In [ ]:
output_file_path = "output.geojson"
with open(output_file_path, 'w') as geojson_file:
    json.dump(response_json, geojson_file, indent=2)
print(f"GeoJSON data has been saved to {output_file_path}")

## Displaying Metadata

The following cell displays metadata for the first 5 and last 5 items retrieved from the Streaming-Imagery API. This preview provides a quick overview of the key information available in the dataset.


In [ ]:
import pandas as pd
df = pd.DataFrame(response_json['features'])
display(df)

# Displaying Search Results on an Interactive Map

We will now display an interactive map to visualize the results retrieved from the Streaming-Imagery API. This map will allow us to explore the geographical boundaries of the search results and examine the metadata associated with each feature that is currently selected.


In [65]:
from ipyleaflet import Map, GeoJSON, basemaps, basemap_to_tiles, Popup, WidgetControl
import ipywidgets as widgets
from IPython.display import display
from shapely.geometry import shape


m = Map(center=(39.87, -105.045), zoom=9)
tile_layer = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
m.add_layer(tile_layer)


geo_json_data = response.json()
features = geo_json_data["features"]
paginated_features = []
page_features_layer = GeoJSON()
metadata_info = widgets.HTML(value="temp")

metadata_box = WidgetControl(widget=metadata_info, position='topright')
m.add_control(metadata_box)

items_per_page = min(len(features), 25)
total_pages = (len(features) + items_per_page - 1) // items_per_page
current_page = 0
current_image = 0

def display_image():
    global metadata_info
    
    selected_feature_layer = GeoJSON(data=paginated_features[current_image], style={"color": "#BD00FF", "fillColor": "#BD00FF", "fillOpacity": 0.5, "weight": 4})
    m.layers = [tile_layer, page_features_layer, selected_feature_layer]

    # Update Metadata Info 
    feature_id = paginated_features[current_image]["id"]
    date = paginated_features[current_image]["properties"]["acquisitionDate"]
    off_nadir_angle = paginated_features[current_image]["properties"]["offNadirAngle"]
    legacy_identifier = paginated_features[current_image]["properties"]["legacyIdentifier"]
    metadata_info.value = f"<div><b>Cat ID: </b>{legacy_identifier}</div> <div><b>Date: </b> {date}</div> <div><b>Off Nadir Angle: </b> {off_nadir_angle}</div>"


def change_page():
    global paginated_features
    global page_features_layer
    start_idx = current_page * items_per_page
    end_idx = min(start_idx + items_per_page, len(features))
    
    paginated_features = features[start_idx:end_idx]
    paginated_geojson_data = {
        "type": "FeatureCollection",
        "features": paginated_features,
    }
    
    page_features_layer = GeoJSON(data=paginated_geojson_data, style={"color": "#BD00FF90", "fillOpacity": 0, "weight": 2})
    m.layers = [tile_layer, page_features_layer]

    


prev_page_button = widgets.Button(description="Previous Page")
next_page_button = widgets.Button(description="Next Page")
prev_image_button = widgets.Button(description="Previous Image")
next_image_button = widgets.Button(description="Next Image")
page_label = widgets.Label()

def on_prev_page_click(b):
    global current_page
    global current_image
    if current_page > 0:
        current_image = 0
        current_page -= 1
    else: 
        current_page = total_pages - 1
    update_display()

def on_next_page_click(b):
    global current_page
    global current_image
    if current_page < total_pages - 1:
        current_image = 0
        current_page += 1
    else:
        current_page = 0
    update_display()

def on_prev_image_click(b):
    global current_image
    if current_image > 0:
        current_image -= 1
        update_display()
    else:
        on_prev_page_click(b)
        current_image = min(len(paginated_features),items_per_page) - 1
        update_display()

def on_next_image_click(b):
    global current_image
    if current_image < min(len(paginated_features),items_per_page) - 1:
        current_image += 1
        update_display()
    else:
        on_next_page_click(b)

def update_display():
    change_page()
    display_image()
    page_label.value = f"Image {current_image + 1} of {min(len(paginated_features),items_per_page)} from Page {current_page + 1} of {total_pages}"

prev_page_button.on_click(on_prev_page_click)
next_page_button.on_click(on_next_page_click)
prev_image_button.on_click(on_prev_image_click)
next_image_button.on_click(on_next_image_click)


update_display()

display(m)
display(widgets.HBox([prev_page_button, prev_image_button, page_label, next_image_button, next_page_button]))


Map(center=[39.87, -105.045], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

# Displaying Feature Image on an Interactive Map
Utilizing the WMS Get Map endpoint from the Streaming-Imagery API, and the Legacy Identifier ID from a selected feature, we are able to display the satelite images that are within the area of the feature. This example shows the result of when a feature's polygon is selected by the user. In sample-app-2 there is a limit of how much you can zoom in without incurring costs. The app has a free mode were it blocks the user from zooming any further.

In [32]:
from ipyleaflet import Map, GeoJSON, basemaps, basemap_to_tiles, WMSLayer
import ipywidgets as widgets
from IPython.display import display


m = Map(center=(39.87, -105.045), zoom=9, zoom_control=False, doubleClickZoom=False, scrollWheelZoom=False)
tile_layer = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
m.add_layer(tile_layer)
m.panes = {
    'wmsPane': {'zIndex': 450},
}

geo_json_data = response.json()
features = geo_json_data["features"]
paginated_features = []
page_features_layer = GeoJSON()

items_per_page = min(len(features), 25)
total_pages = (len(features) + items_per_page - 1) // items_per_page
current_page = 0
current_image = 0
isFreeMode = True

def display_image():
    
    selected_feature_layer = GeoJSON(data=paginated_features[current_image], style={"color": "#BD00FF", "fillColor": "#BD00FF", "fillOpacity": 0.5, "weight": 4})
    m.layers = [tile_layer, page_features_layer, selected_feature_layer]

    wms = WMSLayer(
            url='https://api.maxar.com/streaming/v1/ogc/wms?maxar_api_key='+ maxar_api_key +'&cql_filter=legacyIdentifier%3D%27'+ paginated_features[current_image]["properties"]["legacyIdentifier"] +'%27',
            layers='Maxar:Imagery',
            format='image/png',
            transparent=True,
            pane="wmsPane"
        )
    m.layers = [tile_layer, page_features_layer, selected_feature_layer, wms]

def change_page():
    global paginated_features
    global page_features_layer
    start_idx = current_page * items_per_page
    end_idx = min(start_idx + items_per_page, len(features))
    
    paginated_features = features[start_idx:end_idx]
    paginated_geojson_data = {
        "type": "FeatureCollection",
        "features": paginated_features,
    }
    
    page_features_layer = GeoJSON(data=paginated_geojson_data, style={"color": "#BD00FF90", "fillOpacity": 0, "weight": 2})
    m.layers = [tile_layer, page_features_layer]


zoom_in_button = widgets.Button(description="", icon="plus", tooltip="Zoom In", layout={'width': '30px'})
zoom_out_button = widgets.Button(description="", icon="minus", tooltip="Zoom Out", layout={'width': '30px'})
mode_button = widgets.Button(description="Free Mode", button_style="success")

def zoom_in_handler(b):
    if(mode_button.description=="Free Mode" and m.zoom<14):
        m.zoom += 1
    elif(mode_button.description=="Pay Mode" and m.zoom<18):
        m.zoom += 1
    
def zoom_out_handler(b):
    m.zoom -= 1

def change_mode(b):
    if mode_button.description == "Pay Mode":
        isFreeMode = True
        mode_button.description = "Free Mode"
        if(m.zoom > 14):
            m.zoom = 14
    else:
        isFreeMode = False
        mode_button.description = "Pay Mode"

prev_page_button = widgets.Button(description="Previous Page")
next_page_button = widgets.Button(description="Next Page")
prev_image_button = widgets.Button(description="Previous Image")
next_image_button = widgets.Button(description="Next Image")
page_label = widgets.Label()

def on_prev_page_click(b):
    global current_page
    global current_image
    if current_page > 0:
        current_image = 0
        current_page -= 1
    else: 
        current_page = total_pages - 1
    update_display()

def on_next_page_click(b):
    global current_page
    global current_image
    if current_page < total_pages - 1:
        current_image = 0
        current_page += 1
    else:
        current_page = 0
    update_display()

def on_prev_image_click(b):
    global current_image
    if current_image > 0:
        current_image -= 1
        update_display()
    else:
        on_prev_page_click(b)
        current_image = min(len(paginated_features),items_per_page) - 1
        update_display()

def on_next_image_click(b):
    global current_image
    if current_image < min(len(paginated_features),items_per_page) - 1:
        current_image += 1
        update_display()
    else:
        on_next_page_click(b)

def update_display():
    change_page()
    display_image()
    page_label.value = f"Image {current_image + 1} of {min(len(paginated_features),items_per_page)} from Page {current_page + 1} of {total_pages}"

prev_page_button.on_click(on_prev_page_click)
next_page_button.on_click(on_next_page_click)
prev_image_button.on_click(on_prev_image_click)
next_image_button.on_click(on_next_image_click)

zoom_in_button.on_click(zoom_in_handler)
zoom_out_button.on_click(zoom_out_handler)
mode_button.on_click(change_mode)


update_display()

display(m)
display(widgets.HBox([mode_button, prev_page_button, prev_image_button, page_label, next_image_button, next_page_button]))

zoom_controls = widgets.VBox([zoom_in_button, zoom_out_button])
zoom_control = WidgetControl(widget=zoom_controls, position="topright")
m.add_control(zoom_control)



Map(center=[39.87, -105.045], controls=(AttributionControl(options=['position', 'prefix'], position='bottomrig…